In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

In [4]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset): # build custom dataset
    
    def __init__(self, data, block_size):
        chars = sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = {ch : i for i,ch in enumerate(chars)}
        self.itos = {i : ch for i,ch in enumerate(chars)}
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.block_size + 1]
        # encode every character to an integer
        dix = [self.stoi[s] for s in chunk]
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y
    
    def __len__(self):
        return len(self.data) - self.block_size

In [5]:
block_size = 128 # spatial extent of the modle for itx context

In [6]:
# download text file for training model
from pathlib import Path
from urllib.request import urlretrieve

DATA_PATH = Path("../data")
PATH = DATA_PATH / "gpt"

PATH.mkdir(parents=True, exist_ok=True)

URL = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/"
FILENAME = "input.txt"

if not (PATH / FILENAME).exists():
    urlretrieve(URL + FILENAME, PATH / FILENAME)

with open(PATH / FILENAME, 'r') as f:
    text = f.read()

train_dataset = CharDataset(text, block_size)

data has 1115394 characters, 65 unique.


In [7]:
from model import GPT, GPTConfig
mconf = GPTConfig(train_dataset.vocab_size, block_size,
                 n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

07/25/2021 04:35:52 - INFO - model - number of parameters: 2.535219e+07


In [ ]:
from trainer import Trainer, TrainerConfig
tconf = TrainerConfig(max_epochs=2, batch_size=256, learning_rate=6e-4,
                     lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*block_size,
                     ckpt_path="model/params.ckpt",
                     num_workers=4)
trainer = Trainer(model, train_dataset, None, tconf)
trainer.train()

In [ ]:
from utils import sample

context = "O god, O God!"
x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to(trainer.device)
y = sample(model, x, 2000, temperature=1.0, sample=True, top_k=10)[0]
completion = ''.join([train_dataset.itos[int(i)] for in y])
print(completion)